In [1]:
# Xing Hong xhong37@wisc.edu
# Xingke Sun xsun326@wisc.edu

In [2]:
import pandas as pd
import torch

In [3]:
pd.read_csv

<function pandas.io.parsers.readers.read_csv(filepath_or_buffer: 'FilePath | ReadCsvBuffer[bytes] | ReadCsvBuffer[str]', *, sep: 'str | None | lib.NoDefault' = <no_default>, delimiter: 'str | None | lib.NoDefault' = None, header: "int | Sequence[int] | None | Literal['infer']" = 'infer', names: 'Sequence[Hashable] | None | lib.NoDefault' = <no_default>, index_col: 'IndexLabel | Literal[False] | None' = None, usecols: 'list[HashableT] | Callable[[Hashable], bool] | None' = None, dtype: 'DtypeArg | None' = None, engine: 'CSVEngine | None' = None, converters: 'Mapping[Hashable, Callable] | None' = None, true_values: 'list | None' = None, false_values: 'list | None' = None, skipinitialspace: 'bool' = False, skiprows: 'list[int] | int | Callable[[Hashable], bool] | None' = None, skipfooter: 'int' = 0, nrows: 'int | None' = None, na_values: 'Sequence[str] | Mapping[str, Sequence[str]] | None' = None, keep_default_na: 'bool' = True, na_filter: 'bool' = True, verbose: 'bool' = False, skip_blan

In [4]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train = train.values
test = test.values
trainX = torch.tensor(train[:, :-1], dtype=torch.float64)
trainY = torch.tensor(train[:, -1:], dtype=torch.float64)

testX = torch.tensor(test[:, :-1], dtype=torch.float64)
testY = torch.tensor(test[:, -1:], dtype=torch.float64)

In [5]:
#Q1
trainX_bytes = trainX.shape[0] * trainX.shape[1] * 64 / 8
int(trainX_bytes)

83520

In [6]:
#Q2
trainX_float16 = trainX.to(dtype=torch.float16)
trainX_float64 = trainX_float16.to(dtype=torch.float64)
diff = trainX - trainX_float64
diff = torch.abs(diff)
max_diff = diff.max()
float(max_diff)

0.0

In [7]:
#Q3
torch.cuda.is_available()

False

In [8]:
coef = torch.tensor([
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040], # POS_50_59_CP
        [0.0300], # POS_60_69_CP
        [0.0300],
        [0.0300],
        [0.0300]
], dtype=trainX.dtype)
coef

tensor([[0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0300],
        [0.0300],
        [0.0300],
        [0.0300]], dtype=torch.float64)

In [9]:
#Q4
first_predict = testX[0] @ coef
first_predict.item()

9.844

In [10]:
#Q5
testX_pred = testX @ coef
mean_testX_pred = torch.mean(testX_pred)
mean_testX_pred.item()

12.073632183908048

In [11]:
#Q6
def f(x):
    return x ** 2 - 8 * x + 19
x = torch.tensor(0.0, requires_grad=True)
y = f(x)
float(y)

19.0

In [12]:
#Q7
optimizer = torch.optim.SGD([x], lr=0.1)
for epoch in range(100):
    y = f(x)
    y.backward()
    optimizer.step()
    optimizer.zero_grad()

x.item()

3.999999523162842

In [13]:
#Q8
coef = torch.zeros((10, 1), dtype = torch.float64, requires_grad=True)

X_hat = trainX @ coef
SE = (X_hat - trainY)**2
MSE = SE.sum()/ trainY.shape[0]
MSE.item()

197.8007662835249

In [14]:
#Q9
torch.manual_seed(544)
ds = torch.utils.data.TensorDataset(trainX, trainY)
dl = torch.utils.data.DataLoader(ds, batch_size=50, shuffle=True)
f_loss = torch.nn.MSELoss()
optim  = torch.optim.SGD([coef],lr = 0.000002)

for i in range(500):
    for X,y in dl:
        pred = X @ coef
        loss = f_loss(pred, y)
        loss.backward()
        optim.step()
        optim.zero_grad()


X_hat = trainX @ coef
SE = (X_hat - trainY)**2
MSE = SE.sum()/ trainY.shape[0]
MSE.item()

26.8113940147193

In [15]:
#Q10
X_hat = testX @ coef
SE = (X_hat - testY)**2
MSE = SE.sum()/ testY.shape[0]
MSE.item()

29.05854692548551